In [1]:
def rotateDemo(Stress0, CurStress):
    #================================================================
    # Justin Bonus, September 30, 2019
    # Create stress state that rotates to N-S for animation purposes
    # (3,1) principal normal stress inputs
    # Output: (X,) time, (3,X) rotStress0, (3,X) rotCurStress 
    #================================================================
    Stress0 = dev(Stress0)
    CurStress = dev(CurStress)
    north = dev(np.array([[np.sqrt(3/2)],[0],[0]])) # Deviatoric north unit-vector
    east = dev(np.array([[0],[np.sqrt(1/2)],[-np.sqrt(1/2)]]))
    
    angleNorth = angleNormal(north, Stress0) # Total north angle
    angleEast = angleNormal(east, Stress0)
    if np.isnan(angleNorth):
        angleNorth = 0
    if np.isnan(angleEast):
        angleEast = 0
    if angleEast < np.pi/2:
        angleNorth = -angleNorth # Rotate correct direction
    northSteps = 12
    northStress0 = Stress0 # Initial
    northCurStress = CurStress
    for i in range(0,northSteps):
        angleStep = angleNorth/12
        Stress0 = quatHydroRotator(Stress0, angleStep)
        CurStress = quatHydroRotator(CurStress, angleStep)
        northStress0 = np.append(northStress0,Stress0, axis=1)
        northCurStress = np.append(northCurStress,CurStress, axis=1)        
    
    # Find contour center
    #i = cen_x
    #last = northStress0.shape[1] - 1
    #indexNorthStress0 = stressToIndex(northStress0[:,last], res_x, res_y, R) # Correct?
    #indexNorthCurStress = stressToIndex(northCurStress[:,last], res_x, res_y, R)
    #k = int(indexNorthStress0[1])
    #j = int(indexNorthCurStress[1])
    #kappa = int(kappaM[j,i,k]) # Correct?
    #midpoint = 
    #midpoint = np.array([[midpoint],[],[]])
    midpoint = northStress0[:,-1]/2 # Assumed value, could be found with kappaMatrix
    adjNorthCurStress = (northCurStress[:,-1] - midpoint) # Corrected for contour center
    adjNorthCurStress = adjNorthCurStress.reshape(3,1)

    angleSouth = angleNormal(-north, adjNorthCurStress)
    angleEast = angleNormal(east, adjNorthCurStress)
    if np.isnan(angleSouth):
        angleSouth = 0
    if np.isnan(angleEast):
        angleEast = 0
    if angleEast > np.pi/2:
        angleSouth = -angleSouth
    southSteps = 20 # Frames for south rotation
    southNorthStress0 = northStress0 # Initialize
    southNorthCurStress = northCurStress # Initialize
    for i in range(0,southSteps):
        angleStep = angleSouth/southSteps # Rotation per frame
        southNorthStress0 = np.append(southNorthStress0, northStress0[:,-1].reshape(3,1), axis=1) # Maintain northStress0
        
        # Shift hydrostatic axis for rotation around contour center
        adjNorthCurStress = quatHydroRotator(adjNorthCurStress, angleStep)
        adjAdjNorthCurStress = np.zeros((3,1))
        adjAdjNorthCurStress[0] = adjNorthCurStress[0] + midpoint[0]
        adjAdjNorthCurStress[1] = adjNorthCurStress[1] + midpoint[1]
        adjAdjNorthCurStress[2] = adjNorthCurStress[2] + midpoint[2]

        southNorthCurStress = np.append(southNorthCurStress, adjAdjNorthCurStress, axis=1) # Rotate northCurStress south
    
    buffer = 6 # Extra frames at end
    for i in range(0,buffer):
        southNorthStress0 = np.append(southNorthStress0, southNorthStress0[:,-1].reshape(3,1), axis=1)
        southNorthCurStress = np.append(southNorthCurStress, southNorthCurStress[:,-1].reshape(3,1), axis =1)
        
    rotStress0 = southNorthStress0
    rotCurStress = southNorthCurStress
    time = np.linspace(0,1,rotCurStress.shape[1]+1) # Time increments
    
    return time, rotStress0, rotCurStress